<a href="https://colab.research.google.com/github/pavansai26/FastAPI-microservice-to-classify-the-intents/blob/main/question_and_answer_generation_using_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **# Install the required packages**

In [1]:
!pip install fastapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 4.1 MB/s eta 0:00:00


# **# Import the required libraries**

In [6]:
from fastapi import FastAPI


In [9]:
from IPython.display import clear_output

!pip install transformers==3.3.1

clear_output()

In [16]:
import transformers
from transformers import T5ForConditionalGeneration, T5TokenizerFast

# **The Tokenizer**

In [14]:
checkpoint = "t5-base"
model = T5ForConditionalGeneration.from_pretrained(checkpoint)
tokenizer = T5TokenizerFast.from_pretrained(checkpoint)

In [15]:

# Initialize the FastAPI app
app = FastAPI()

# Define the /qa endpoint
@app.post('/qa')
def generate_qa(text: str):
    # Split the text into paragraphs
    paragraphs = text.split('\n\n')
    
    # Initialize the list of questions and answers
    questions = []
    answers = []
    
    # Loop over each paragraph
    for paragraph in paragraphs:
        # Generate a question for the paragraph
        input_text = "generate question: " + paragraph
        input_ids = tokenizer.encode(input_text, return_tensors='pt')
        output = model.generate(input_ids=input_ids, max_length=32, num_beams=4, early_stopping=True)
        question = tokenizer.decode(output[0], skip_special_tokens=True)
        
        # Generate an answer for the question
        input_text = "answer: " + question + " context: " + paragraph
        input_ids = tokenizer.encode(input_text, return_tensors='pt')
        output = model.generate(input_ids=input_ids, max_length=256, num_beams=4, early_stopping=True)
        answer = tokenizer.decode(output[0], skip_special_tokens=True)
        
        # Add the question and answer to the lists
        questions.append(question)
        answers.append(answer)
    
    # Return the list of questions and answers
    return {'questions': questions, 'answers': answers}


# **You can then start the server by running:**

In [ ]:
uvicorn main:app --reload


# **This will start the server at http://localhost:8000. You can then send a POST request to the /qa endpoint with the text of your product manual in the request body, like this:**

In [ ]:
import requests

text = """
This is the first paragraph. It contains some information about the product.

This is the second paragraph. It contains some more information about the product.
"""

response = requests.post('http://localhost:8000/qa', json={'text': text})
print(response.json())
